In [1]:
from lc_correction.compute import *

import pandas as pd
import numpy as np

In [2]:
detections = pd.read_parquet("../data_examples/parquets/10_objects_detections.parquet")
detections["mjd"] = detections.jd - 2400000.5
del detections["jd"]

display(detections.head())

,objectId,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,candid,isdiffpos,tblid,...,szmag3,sgscore3,distpsnr3,nmtchps,rfid,jdstartref,jdendref,nframesref,parent_candid,mjd
0,ZTF18aaioozo,1,518415875315,20.182840,ztf_20180603415822_000683_zg_c14_o_q2_scimrefd...,Kulkarni,1,518415875315010002,f,2,...,15.798900,0.998750,8.699934,15,683120153,2.458205e+06,2.458231e+06,15,0,58272.415880
1,ZTF18aaurrwb,1,524348253515,20.939032,ztf_20180609348229_000723_zg_c09_o_q4_scimrefd...,Kulkarni,1,524348253515015051,t,51,...,21.497999,0.751875,19.763674,9,723120135,2.458204e+06,2.458207e+06,16,0,58278.348252
2,ZTF18abadmxd,1,527431606115,20.560465,ztf_20180612431609_000804_zg_c16_o_q2_scimrefd...,Kulkarni,1,527431606115010124,f,124,...,16.329300,0.974434,8.267725,37,804120161,2.458229e+06,2.458277e+06,15,0,58281.431609
3,ZTF18abadvod,2,527448113315,20.925859,ztf_20180612448102_000687_zr_c09_o_q2_scimrefd...,Kulkarni,1,527448113315010023,f,23,...,20.278700,0.088690,4.907485,53,687120233,2.458199e+06,2.458257e+06,15,0,58281.448113
4,ZTF18abadvtf,2,527450573615,20.881939,ztf_20180612450556_000730_zr_c10_o_q1_scimrefd...,Kulkarni,1,527450573615015012,t,12,...,20.271200,0.625548,7.855210,34,730120236,2.458223e+06,2.458280e+06,15,0,58281.450579


In [3]:
correction_df = detections.groupby(["objectId", "fid"]).apply(apply_correction_df)
correction_df.reset_index(inplace=True)

correction_df["step_id_corr"] = "test"
correction_df["has_stamp"] = correction_df["parent_candid"] == 0
display(correction_df.head())

,objectId,fid,candid,pid,diffmaglim,pdiffimfilename,programpi,programid,isdiffpos,tblid,...,parent_candid,mjd,corrected,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,dubious,mjdendref,step_id_corr,has_stamp
0,ZTF17aaaecgt,1,550436093915015009,550436093915,20.544632,ztf_20180705436100_000801_zg_c10_o_q4_scimrefd...,Kulkarni,1,1.0,9,...,0,58304.436099,True,16.510900,0.014875,0.018298,False,58288.407836,test,True
1,ZTF17aaaecgt,1,1209408473915010011,1209408473915,20.269983,ztf_20200424408472_000801_zg_c10_o_q4_scimrefd...,Kulkarni,1,-1.0,11,...,0,58963.408472,True,17.879532,0.055861,0.067333,False,58288.407836,test,True
2,ZTF17aaaecgt,1,1004203723915015017,1004203723915,20.481764,ztf_20191002203727_000801_zg_c10_o_q4_scimrefd...,TESS,1,1.0,17,...,0,58758.203727,True,16.742756,0.009440,0.016224,False,58288.407836,test,True
3,ZTF17aaaecgt,1,978289493915010013,978289493915,20.881487,ztf_20190906289468_000801_zg_c10_o_q4_scimrefd...,TESS,1,-1.0,13,...,0,58732.289491,True,17.739686,0.052527,0.062059,False,58288.407836,test,True
4,ZTF17aaaecgt,1,1009243913915015013,1009243913915,20.051744,ztf_20191007243912_000801_zg_c10_o_q4_scimrefd...,TESS_Prince,3,1.0,13,...,0,58763.243912,True,16.697498,0.016639,0.020905,False,58288.407836,test,True


In [10]:
correction_df.to_parquet("10_objects_corrected.parquet")

In [4]:
magstats = correction_df.groupby(["objectId", "fid"]).apply(apply_mag_stats)
magstats.reset_index(inplace=True)

/home/javier/anaconda3/envs/lc_correction/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/javier/anaconda3/envs/lc_correction/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/javier/anaconda3/envs/lc_correction/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [5]:
magstats

,objectId,fid,corrected,nearZTF,nearPS1,stellarZTF,stellarPS1,stellar,ndet,ndubious,...,magap_median,magap_max,magap_min,sigmap,magap_first,magap_last,first_mjd,last_mjd,flag_saturation,flag_reference
0,ZTF17aaaecgt,1,True,True,True,True,True,True,231,0,...,18.237900,20.348900,17.119900,0.672256,18.382500,18.085199,58289.408912,59009.280961,0.0,False
1,ZTF17aaaecgt,2,True,True,True,True,True,True,208,3,...,16.024151,17.957899,14.922700,0.736437,16.830000,16.696699,58303.363044,59004.307280,0.0,True
2,ZTF18aaioozo,1,True,True,True,True,True,True,98,0,...,17.797699,19.234100,17.003799,0.414520,18.709400,18.286301,58244.431273,59009.276736,0.0,True
3,ZTF18aaioozo,2,True,True,True,True,True,True,60,0,...,17.736549,18.667200,17.453600,0.257676,17.982201,17.679399,58244.462130,58970.409595,0.0,True
4,ZTF18aaurrwb,1,False,False,False,True,False,False,3,0,...,20.370001,20.608601,20.096800,0.256095,20.608601,20.096800,58258.374213,58278.362986,NaN,True
5,ZTF18aazeood,1,True,True,True,True,True,True,367,0,...,18.181900,19.944500,17.461500,0.441500,17.979200,18.506001,58278.410035,59010.477998,0.0,False
6,ZTF18aazeood,2,True,True,True,True,True,True,396,0,...,17.561350,19.102301,16.823500,0.350970,17.434200,18.278900,58289.452558,59010.350660,0.0,False
7,ZTF18abadmxd,1,True,True,True,True,True,True,3,0,...,18.576799,19.036900,18.224001,0.407628,18.576799,18.224001,58281.431609,58320.457442,0.0,True
8,ZTF18abadvod,1,True,True,True,True,True,True,1,0,...,20.612000,20.612000,20.612000,NaN,20.612000,20.612000,58390.151840,58390.151840,0.0,True
9,ZTF18abadvod,2,True,True,True,True,True,True,7,0,...,19.937099,20.225201,19.723200,0.176412,20.225201,19.723200,58281.448113,58669.405718,0.0,True


In [6]:
objstats = apply_object_stats_df(correction_df, magstats)
objstats.reset_index(inplace=True)

In [8]:
objstats[["flag_diffpos"]]

,flag_diffpos
0,False
1,False
2,True
3,False
4,False
5,False
6,True
7,False
8,False
9,True


In [ ]:
non_det = pd.read_parquet("../data_examples/parquets/10_objects_non_detections.parquet")

In [ ]:
non_det["mjd"] = non_det.jd - 2400000.5
del non_det["jd"]

In [ ]:
dd = do_dmdt_df(magstats, non_det)
magstats["step_id_corr"] = "test"

In [ ]:
magstats = magstats.join(dd, on=["objectId", "fid"])
magstats

In [ ]:
magstats[["objectId", "fid", "first_mjd", "last_mjd", "ndubious"]]

In [ ]:
def get_flag_saturation(x, fid):
    x = x[x.corrected]
    total = x[x["fid"] == fid]['magpsf_corr'].count()
    if total == 0:
        return np.nan
    satured = (x[x["fid"] == fid]["magpsf_corr"] < 13.2).sum()
    return satured/total

def get_flag_reference(detections, fid, magstat):
    detections = detections[detections["fid"] == fid]
    if len(detections) == 0:
        return np.nan
    first_detection = magstat[magstat["fid"] == fid]["first_mjd"].values[0]
    last_reference = detections["mjdendref"].max()
    return last_reference < first_detection

def add_flags(magstat, det, objstats, index_name="objectId"):
    flags = pd.DataFrame()
    flags["flag_ref"] = det.groupby(index_name).mjdendref.apply(max).values < objstats.firstmjd # rehacer obj x banda
    
    flags["flag_diffpos"] = det.groupby(index_name).isdiffpos.min().values > 0 # nice obj x da lo mismo dijo el camilio

    
    flags["flag_saturation_g"] = det.groupby(index_name).apply(lambda x: get_saturation(x, 1)).values
    flags["flag_saturation_r"] = det.groupby(index_name).apply(lambda x: get_saturation(x, 2)).values
    return flags

def flags_by_fid(det, magstat, index_name="objectId"):
    oid = det[index_name].values[0]
    magstat = magstat[magstat[index_name] == oid]
    flags = {}
    flags["flag_saturation_g"] = get_flag_saturation(det, 1)
    flags["flag_saturation_r"] = get_flag_saturation(det, 2)
    flags["flag_ref_g"] = get_flag_reference(det, 1, magstat)
    flags["flag_ref_r"] = get_flag_reference(det, 2, magstat)
    return pd.Series(flags)

In [ ]:
correction_df.groupby("objectId").apply(flags_by_fid, magstat=magstats)

In [ ]:
correction_df[["objectId", "rfid", "fid", "mjd", "mjdendref"]][(correction_df.objectId == "ZTF17aaaecgt") & (correction_df.parent_candid == 0)]

In [ ]:
np.nan < 3